# Segmenting and Clustering Neighborhoods in Toronto


<b><font size ="3" color="red">Note: </font> <font size ="3" >I have made single notebook for all three question please scroll down for all review all questions</font></b>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Question 1: Create dataframe</a>

2. <a href="#item2">Question 2: Load longitude and latitude values</a>

3. <a href="#item3">Question 3: Explore and cluster the neighborhoods</a>
 
</font>
</div>

<p id="item1"> </p>

## Question 1: Create dataframe
Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

### Requirements:
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11** in the above table.
- If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.

#### Install Beautiful Soup 
Beautiful Soup is a library that makes it easy to scrape information from web pages. It sits atop an HTML or XML parser, providing Pythonic idioms for iterating, searching, and modifying the parse tree.

In [1]:
# !conda install -c anaconda beautifulsoup4

#### Load data form Wipipedia Website by Beautiful Soup 

In [2]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urllib.request.urlopen(url).read()

In [4]:
soup = BeautifulSoup(html, 'html.parser')

In [5]:
table = soup.find('table')

In [6]:
row_data = []

for tr in table.find_all('tr'):
    cell_data = []
    
    for td in tr.find_all('td'):
        cell_data.append(td.text.strip())
        
    if len(cell_data) == 3:
        row_data.append(cell_data)

#### Tranform the data into a *pandas* dataframe

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(row_data, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Data cleaning

In [10]:
df=df[df['Borough']!='Not assigned']

In [11]:
df['Neighborhood'].replace('Not assigned',df['Borough'],inplace=True)

In [12]:
result = df.groupby(['PostalCode','Borough'], sort=False).agg( ', '.join)

In [13]:
df_neigh = result.reset_index()
df_neigh.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_neigh['Borough'].unique()),
        df_neigh.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


<p id="item2"> </p>

## Question 2: Load longitude and latitude values
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [15]:
# !conda install -c conda-forge geocoder

##### Example

```python
import geocoder

postal_code = 'M3A'

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```

In [16]:
df_post = pd.read_csv("http://cocl.us/Geospatial_data")

In [17]:
df_post.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_post.set_index("PostalCode")
df_neigh.set_index("PostalCode")
toronto_data=pd.merge(df_neigh, df_post)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


<p id="item3"> </p>

## Question 3: Explore and cluster the neighborhoods

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.

#### Use geopy library to get the latitude and longitude values of Torronto

In [18]:
# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

In [19]:
address = 'Toronto, ON, Canada'

# geolocator = Nominatim(user_agent="to_explorer") # GeocoderTimedOut: Service timed out
# location = geolocator.geocode(address)
# latitude = location.latitude
# longitude = location.longitude

latitude = 43.653963
longitude = -79.387207
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

In [20]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [21]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### Define Foursquare Credentials and Version

In [22]:
import requests

##### Explore Neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Create a new dataframe called manhattan_venues

In [25]:
torronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Queen's Park
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riv

##### Size of the resulting dataframe

In [26]:
print(torronto_venues.shape)
torronto_venues.head()

(2222, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


##### Venues for each neighborhood

In [27]:
torronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",10,10,10,10,10,10


##### Numbers of unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(torronto_venues['Venue Category'].unique())))

There are 266 uniques categories.


##### Analyze Each Neighborhood

In [29]:
# one hot encoding
torronto_onehot = pd.get_dummies(torronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torronto_onehot['Neighborhood'] = torronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torronto_onehot.columns[-1]] + list(torronto_onehot.columns[:-1])
torronto_onehot = torronto_onehot[fixed_columns]

torronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
torronto_onehot.shape

(2222, 266)

##### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
torronto_grouped = torronto_onehot.groupby('Neighborhood').mean().reset_index()
torronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.020000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.01
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.050000,0.000000,0.000000,0.000000,0.000000,0.0,0.00
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.017857,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00


In [32]:
torronto_grouped.shape

(99, 266)

##### Print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in torronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = torronto_grouped[torronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.04
2   Steakhouse  0.04
3          Bar  0.04
4   Restaurant  0.03


----Agincourt----
                       venue  freq
0                     Lounge   0.2
1  Latin American Restaurant   0.2
2               Skating Rink   0.2
3             Clothing Store   0.2
4             Breakfast Spot   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0          Playground  0.33
1                Park  0.33
2    Sculpture Garden  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.22
1           Pizza Place  0.11
2   Fried Chicken Joint  0.11
3        Sandwich Place  0.11
4  Fast Food Restaurant  0.11


----Alderwood, Long Branch----
                venue  freq
0         Pizza

                        venue  freq
0                      Bakery  0.25
1  Construction & Landscaping  0.25
2                        Park  0.25
3            Basketball Court  0.25
4          Mexican Restaurant  0.00


----East Birchmount Park, Ionview, Kennedy Park----
              venue  freq
0    Discount Store  0.33
1  Department Store  0.17
2       Bus Station  0.17
3        Hobby Shop  0.17
4       Coffee Shop  0.17


----East Toronto----
                       venue  freq
0                       Park  0.50
1                Coffee Shop  0.25
2          Convenience Store  0.25
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Emery, Humberlea----
                             venue  freq
0           Furniture / Home Store  0.33
1    Paper / Office Supplies Store  0.33
2                   Baseball Field  0.33
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----Fairview, Henry Farm, Oriole----
                  venue  freq

                             venue  freq
0                       Playground   0.5
1                Convenience Store   0.5
2                      Yoga Studio   0.0
3               Mexican Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----Silver Hills, York Mills----
                             venue  freq
0                             Park   1.0
1                      Yoga Studio   0.0
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----St. James Town----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2   Restaurant  0.05
3       Bakery  0.03
4        Hotel  0.03


----Stn A PO Boxes 25 The Esplanade----
                 venue  freq
0          Coffee Shop  0.12
1                 Café  0.04
2                Hotel  0.03
3   Italian Restaurant  0.03
4  Japanese Restaurant  0.03


----Studio District----
                venue  freq
0                Café  0.10
1         Coffee Shop  0

##### Top 10 venues for each neighborhood.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
import numpy as np

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torronto_grouped['Neighborhood']

for ind in np.arange(torronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Café,Bar,Burger Joint,Hotel,Thai Restaurant,Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Sculpture Garden,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Liquor Store,Pharmacy,Pizza Place,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Sandwich Place,Discount Store,Dessert Shop
4,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Skating Rink,Pharmacy,Sandwich Place,Athletics & Sports,Pool,Pub,Dim Sum Restaurant


##### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [37]:
from sklearn.cluster import KMeans

In [38]:
# set number of clusters
kclusters = 5

torronto_grouped_clustering = torronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 3, 3, 0, 3, 0, 0, 0])

##### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torronto_merged = toronto_data
torronto_merged = torronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

torronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Bus Stop,Park,Food & Drink Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,French Restaurant,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Dessert Shop,Chocolate Shop
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Event Space,Coffee Shop,Shoe Store,Miscellaneous Shop,Carpet Store,Furniture / Home Store,Boutique,Gift Shop,Accessories Store
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Gym,Park,Diner,Portuguese Restaurant,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop


In [40]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Visualize the resulting clusters

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torronto_merged['Latitude'], torronto_merged['Longitude'], torronto_merged['Neighborhood'], torronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    if np.isnan(cluster):
        cluster = np.nan_to_num(cluster)

    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [42]:
torronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Bus Stop,Park,Food & Drink Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,French Restaurant,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Dessert Shop,Chocolate Shop
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Event Space,Coffee Shop,Shoe Store,Miscellaneous Shop,Carpet Store,Furniture / Home Store,Boutique,Gift Shop,Accessories Store
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Gym,Park,Diner,Portuguese Restaurant,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop


##### Examine Cluster 1

In [43]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 0, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,French Restaurant,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,Downtown Toronto,0.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Dessert Shop,Chocolate Shop
3,North York,0.0,Clothing Store,Event Space,Coffee Shop,Shoe Store,Miscellaneous Shop,Carpet Store,Furniture / Home Store,Boutique,Gift Shop,Accessories Store
4,Downtown Toronto,0.0,Coffee Shop,Gym,Park,Diner,Portuguese Restaurant,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop
5,Queen's Park,0.0,Coffee Shop,Gym,Park,Diner,Portuguese Restaurant,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop
9,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Lingerie Store,Ramen Restaurant,Bakery,Sporting Goods Shop
12,Scarborough,0.0,Golf Course,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
13,North York,0.0,Gym,Asian Restaurant,Clothing Store,Beer Store,Coffee Shop,Dim Sum Restaurant,Bike Shop,Discount Store,Sporting Goods Shop,Chinese Restaurant
14,East York,0.0,Park,Cosmetics Shop,Skating Rink,Beer Store,Bus Stop,Asian Restaurant,Curling Ice,Pharmacy,Video Store,Comic Shop
15,Downtown Toronto,0.0,Coffee Shop,Restaurant,Café,Cocktail Bar,Hotel,Cosmetics Shop,Bakery,Beer Bar,Breakfast Spot,Seafood Restaurant


##### Examine Cluster 2

In [44]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 1, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,1.0,Convenience Store,Playground,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


##### Examine Cluster 3

In [45]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 2, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,2.0,Tennis Court,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store,Deli / Bodega


##### Examine Cluster 4

In [46]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 3, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3.0,Fast Food Restaurant,Women's Store,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
7,North York,3.0,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
8,East York,3.0,Pizza Place,Fast Food Restaurant,Pet Store,Breakfast Spot,Gym / Fitness Center,Café,Pharmacy,Gastropub,Bank,Bus Line
10,North York,3.0,Bakery,Pizza Place,Japanese Restaurant,Pub,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
18,Scarborough,3.0,Spa,Electronics Store,Pizza Place,Mexican Restaurant,Breakfast Spot,Rental Car Location,Intersection,Medical Center,Women's Store,Discount Store
39,North York,3.0,Bank,Chinese Restaurant,Japanese Restaurant,Café,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
50,North York,3.0,Pizza Place,Empanada Restaurant,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
60,North York,3.0,Grocery Store,Liquor Store,Gym / Fitness Center,Athletics & Sports,Discount Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
63,York,3.0,Grocery Store,Pizza Place,Caribbean Restaurant,Breakfast Spot,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
70,Etobicoke,3.0,Chinese Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Discount Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run


##### Examine Cluster 5

In [47]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 4, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Bus Stop,Park,Food & Drink Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
21,York,4.0,Park,Women's Store,Fast Food Restaurant,Market,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
35,East York,4.0,Park,Convenience Store,Coffee Shop,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
40,North York,4.0,Park,Snack Place,Airport,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore
45,North York,4.0,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
61,Central Toronto,4.0,Park,Bus Line,Swim School,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Event Space
64,York,4.0,Park,Convenience Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
66,North York,4.0,Park,Bank,Convenience Store,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
85,Scarborough,4.0,Park,Playground,Sculpture Garden,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
91,Downtown Toronto,4.0,Park,Trail,Playground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
